# Use AutoML training video object tracking model for batch prediction 


# Overview


This project demonstrates how to use the Vertex AI SDK for Python to create video object tracking models and do batch prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

### Objective

- Create an AutoML video object tracking model from a Python script
- Create a Vertex `Dataset` resource.
- Train the model.
- View the model evaluation.
- Make a batch prediction.

### Dataset

The dataset used for this tutorial is the [Traffic](https://storage.googleapis.com/automl-video-demo-data/traffic_videos/traffic_videos_labels.csv) dataset. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket.

#### Execute notebook in Colab
<a href="https://colab.research.google.com/github/paulycloud/ml_portfolio/blob/main/AutoML/01_image_classification/02_salad_categories/index.ipynb">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
</a>

## Installation

Install the latest version of Vertex SDK for Python.

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.8 MB/s 
     |████████████████████████████████| 47 kB 4.3 MB/s 
     |████████████████████████████████| 408 kB 58.6 MB/s 
     |████████████████████████████████| 106 kB 51.6 MB/s 
     |████████████████████████████████| 115 kB 55.4 MB/s 
     |████████████████████████████████| 233 kB 55.5 MB/s 
     |████████████████████████████████| 106 kB 50.8 MB/s 
     |████████████████████████████████| 107 kB 37.8 MB/s 
     |████████████████████████████████| 107 kB 53.9 MB/s 
     |████████████████████████████████| 107 kB 46.3 MB/s 
     |████████████████████████████████| 106 kB 53.4 MB/s 
     |████████████████████████████████| 106 kB 55.4 MB/s 
     |████████████████████████████████| 106 kB 51.1 MB/s 
     |████████████████████████████████| 106 kB 48.3 MB/s 
     |████████████████████████████████| 105 kB 53.1 MB/s 
     |████████████████████████████

Install the latest GA version of *google-cloud-storage* library as well.

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

## Project variables 
These are the project variable used in this ML Model: 

In [ ]:
PROJECT_ID = "" # @param {type:"string"}
automl_type = "video" # @param {type:"string"}
model_display_name = "automl_video_traffic_detection_model"
job_display_name = model_display_name + "_job"
endpoint_display_name = model_display_name + "_endpoint"
dataset_display_name = model_display_name + "_dataset"
# bucket details
BUCKET_NAME = "auto-ml-tutorials"
BUCKET_URI = f"gs://{BUCKET_NAME}/{automl_type}/"
OUTPUTBUCKET = f"gs://auto_ml_datasets_predictions/{automl_type}/"

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID
print(PROJECT_ID)

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"
if REGION == "[your-region]":
    REGION = "us-central1"

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

         0  2022-07-23T20:52:09Z  gs://auto-ml-tutorials/video/#1658609529122789  metageneration=1
     54106  2022-07-23T20:52:58Z  gs://auto-ml-tutorials/video/hmdb_split1_5classes_all.csv#1658609578821711  metageneration=1
TOTAL: 2 objects, 54106 bytes (52.84 KiB)


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import json
import os

import google.cloud.aiplatform as aiplatform
from google.cloud import storage

## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# Tutorial

Now you are ready to start creating your own AutoML video object tracking model.

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/ai-platform-unified/video/traffic/traffic_videos_labels.csv"

#### Quick peek at your data

This tutorial uses a version of the Traffic dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

! gsutil cat $FILE | head

Number of Examples 685
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,sedan,1565750291672021,11.933333,0.509205,0.594283,,,0.728737,0.760959,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672171,17.566666,0.761241,0.498466,,,0.948839,0.668524,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672223,20.433333,0.000000,0.465235,,,0.142638,0.665644,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672347,25.766666,0.486523,0.592331,,,0.720611,0.776687,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672575,28.966666,0.578534,0.652778,,,0.828647,0.862967,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672549,28.966666,0.000000,0.518571,,,0.148841,0.737677,,
gs://automl-video-demo-data/traffic_videos/highway_005.mp4,pickup_suv_van,1565750291672599,28.966666,0.106979,0.458078,,,0.377877,0.678937,,

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `VideoDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.

This operation may take several minutes.

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name=dataset_display_name,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.object_tracking,
)

print(dataset.resource_name)

Creating VideoDataset


INFO:google.cloud.aiplatform.datasets.dataset:Creating VideoDataset


Create VideoDataset backing LRO: projects/993987777814/locations/us-central1/datasets/928500186261487616/operations/6799671620345528320


INFO:google.cloud.aiplatform.datasets.dataset:Create VideoDataset backing LRO: projects/993987777814/locations/us-central1/datasets/928500186261487616/operations/6799671620345528320


VideoDataset created. Resource name: projects/993987777814/locations/us-central1/datasets/928500186261487616


...

Import VideoDataset data backing LRO: projects/993987777814/locations/us-central1/datasets/928500186261487616/operations/6428124651087462400


INFO:google.cloud.aiplatform.datasets.dataset:Import VideoDataset data backing LRO: projects/993987777814/locations/us-central1/datasets/928500186261487616/operations/6428124651087462400


VideoDataset data imported. Resource name: projects/993987777814/locations/us-central1/datasets/928500186261487616


INFO:google.cloud.aiplatform.datasets.dataset:VideoDataset data imported. Resource name: projects/993987777814/locations/us-central1/datasets/928500186261487616


projects/993987777814/locations/us-central1/datasets/928500186261487616


### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLVideoTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A video classification model.
  - `object_tracking`: A video object tracking model.
  - `action_recognition`: A video action recognition model.


In [ ]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name=job_display_name,
    prediction_type="object_tracking",
)

print(job)

#### Run the training pipeline

Next, you start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto 5 hours.

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6035150880263634944?project=993987777814


INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6035150880263634944?project=993987777814


AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


...

INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


INFO:google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944 current state:
PipelineState.PIPELINE_STATE_RUNNING


## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

First, you need to get a reference to the new model. As with datasets, you can either use the reference to the model variable you created when you deployed the model or you can list all of the models in your project.

In [ ]:
# Get model resource ID
models = aiplatform.Model.list(filter="display_name=" + model_display_name)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{endpoint_display_name}"}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

## Send a batch prediction request

Send a batch prediction to your deployed model.

### Get test item(s)

Now do a batch prediction to your Vertex model. You will use arbitrary examples out of the dataset as a test items. Don't be concerned that the examples were likely used in training the model -- we just want to demonstrate how to make a prediction.

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2
cols_1 = test_items[0].split(",")
cols_2 = test_items[1].split(",")
if len(cols_1) > 12:
    test_item_1 = str(cols_1[1])
    test_item_2 = str(cols_2[1])
    test_label_1 = str(cols_1[2])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_item_2 = str(cols_2[0])
    test_label_1 = str(cols_1[1])
    test_label_2 = str(cols_2[1])

### Make a batch input file

Now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each video. The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the video.
- `mimeType`: The content type. In our example, it is a `avi` file.
- `timeSegmentStart`: The start timestamp in the video to do prediction on. *Note*, the timestamp must be specified as a string and followed by s (second), m (minute) or h (hour).
- `timeSegmentEnd`: The end timestamp in the video to do prediction on.

In [ ]:
test_filename = "test.jsonl"
gcs_input_uri = BUCKET_URI + "/test.jsonl"
# making data_1 and data_2 variables using the structure mentioned above
data_1 = {
    "content": test_item_1,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

data_2 = {
    "content": test_item_2,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

# getting reference to bucket
bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI.replace("gs://", ""))

# creating a blob
blob = bucket.blob(blob_name=test_filename)

# creating data variable
data = json.dumps(data_1) + "\n" + json.dumps(data_2) + "\n"

# uploading data variable content to bucket
blob.upload_from_string(data)

# printing path of uploaded file
print(gcs_input_uri)

# printing content of uploaded file
! gsutil cat $gcs_input_uri

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="batch_" + job_display_name,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, one can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [ ]:
batch_predict_job.wait()

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `content`: The prediction request.
- `prediction`: The prediction response.
 - `id`: The internal assigned unique identifiers for each prediction request.
 - `displayName`: The class names for the predicted label.
 - `confidences`: The predicted confidence, between 0 and 1, per class label.
 - `timeSegmentStart`: The time offset in the video to the start of the video sequence.
 - `timeSegmentEnd`: The time offset in the video to the end of the video sequence.
 - `frames`: Location with frames of the tracked object.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}".replace(
        BUCKET_URI + "/", ""
    )
    data = bucket.get_blob(gfile_name).download_as_string()
    data = json.loads(data)
    print(data)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI